# Chapter 3 : Fine-Tuning BERT Models

Previously we defined the building blocks of the architecture of the original Transformer. Think of the original Transformer as a model built with LEGO® bricks. The construction set contains bricks such as encoders, decoders, embedding layers, positional encoding methods, multi-head attention layers, masked multi-head attention layers, post-layer normalization, feed-forward sub-layers, and linear output layers.

The bricks come in various sizes and forms. You can spend hours building all sorts of models using the same building kit! Some constructions will only require some of the bricks. Other constructions will add a new piece, just like when we obtain additional bricks for a model built using LEGO® components.

**BERT added a new piece to the Transformer building kit: a bidirectional multi-head attention sub-layer**. When we humans have problems understanding a sentence, we do not just look at the past words. **BERT, like us, looks at all the words in the same sentence at the same time**.

This chapter will first explore the architecture of Bidirectional Encoder Representations from Transformers (BERT). **BERT only uses the blocks of the encoders of the Transformer in a novel way and does not use the decoder stack**.

Then we will fine-tune a pretrained BERT model. The BERT model we will fine-tune was trained by a third party and uploaded to Hugging Face. Transformers can be pretrained. Then, a pretrained BERT, for example, can be fine-tuned on several NLP tasks

## **The architecture of BERT**

BERT introduces **bidirectional attention** to transformer models. Bidirectional attention requires many other changes to the original Transformer model.

We will focus on the evolutions designed by *Devlin* et al. (2018), which describe the encoder stack. We will first go through the encoder stack, then the preparation of the pretraining input environment. Then we will describe the two-step framework of BERT: pretraining and fine-tuning.

Let’s first explore the encoder stack.


### **The encoder stack**

The first building block we will take from the original Transformer model is an encoder layer. The encoder layer, as described in the previous chapter.

![Alt text](encoder_layer.png)

**<font color=yellow>The BERT model does not use decoder layers</font>**. **A BERT model has an encoder stack but no decoder stacks**. **The masked tokens (hiding the tokens to predict) are in the attention layers of the encoder**, as we will see when we zoom into a BERT encoder layer in the following sections.

The original Transformer contains a stack of $N=6$ layers. The number of dimensions of the original Transformer is $d_{model} = 512$. The number of attention heads of the original Transformer is $A=8$. The dimensions of the head of the original Transformer are:

$$
d_k = \frac{d_{model}}{A} = \frac{512}{8} = 64
$$

BERT encoder layers are larger than the original Transformer model.
$2$ BERT models can be built with the encoder layers:

* **$BERT_{BASE}$**, which contains a stack of $N=12$ encoder layers. $d_{model} = 768$ and can also be expressed as $H=768$, as in the BERT paper. **A multi-head attention sub-layer contains $A=12$ heads**. The dimension of each head $z_A$ remains $64$ as in the original Transformer model:
$$
d_k = \frac{d_{model}}{A} = \frac{768}{12} = 64
$$
* **The output of each multi-head attention sub-layer** before concatenation will be the output of the $12$ heads:
$$
output\_multi\_head\_attention=\{z_0, z_1, z_2,\dots, z_{11}\}
$$
* **$BERT_{LARGE}$** which contains a stack of $N=24$ encoder layers. $d_{model} = 1024$. A multi-head attention sub-layer contains $A=16$ heads. The dimension of each head $z_A$ also remains $64$ as in the original Transformer model:
$$
d_k = \frac{d_{model}}{A} = \frac{1024}{16} = 64
$$
* **The output of each multi-head attention sub-layer** before concatenation will be the output of the $16$ heads:
$$
output\_multi\_head\_attention=\{z_0, z_1, z_2,\dots, z_{15}\}
$$
The sizes of the models can be summed up as follows:

![Alt text](transformers_models.png)

> BERT models are not limited to these three configurations. These three configurations illustrate the main aspects of BERT models. Numerous variations are possible.

Size and dimensions play an essential role in BERT-style pretraining. BERT models are like humans. **BERT models produce better results with more working memory (dimensions) and more knowledge (data)**. Large transformer models that learn large amounts of data will pretrain better for downstream NLP tasks.

Let’s go to the first sub-layer and see the fundamental aspects of **input embedding** and **positional encoding** in a BERT model.

## **Preparing the pretraining input environment**

The BERT model has no decoder stack of layers. As such, **it does not have a masked multi-head attention sub-layer**. BERT designers state that **a masked multi-head attention layer that masks the rest of the sequence impedes the attention process**.

A masked multi-head attention layer masks all of the tokens that are beyond the present position. For example, take the following sentence:

```bash
The cat sat on it because it was a nice rug.
```

If we have just reached the word ***it***, the input of the encoder could be:
```bash
The cat sat on it <masked sequence>
```

The motivation of this approach is **to prevent the model from seeing the output it is supposed to predict**. This **left-to-right approach produces relatively good results**.

However, the model cannot learn much this way. To know what ***it*** refers to, we need to see the whole sentence to reach the word ***rug*** and figure out that ***it*** was the rug.

The authors of BERT came up with an idea. **Why not pretrain the model to make predictions using a different approach?**

The authors of BERT came up with **bidirectional attention**, **letting an attention head attend to all of the words both from left to right and right to left**. In other words, **the self-attention mask of an encoder could do the job without being hindered by the masked multi-head attention sub-layer of the decoder**.

The model was trained with two tasks. The first method is **Masked Language Modeling (MLM)**. The second method is **Next Sentence Prediction (NSP)**.

### **Masked language modeling**

Masked language modeling does not require training a model with a sequence of visible words followed by a masked sequence to predict.

BERT introduces the *bidirectional analysis* of a sentence with a random mask on a word of the sentence.

It is important to note that BERT applies **WordPiece**, **a subword segmentation tokenization method**, to the inputs. It also uses **learned positional encoding**, not the sine-cosine approach.

A potential input sequence could be:
```bash
The cat sat on it because it was a nice rug.
```

The **decoder would mask the attention sequence after the model reached the word** ***it***:

```bash
The cat sat on it <masked sequence>.
```

But the **BERT encoder masks a random token to make a prediction**:
```bash
The cat sat on it [MASK] it was a nice rug.
```

The **multi-attention sub-layer can now see the whole sequence, run the self-attention process, and predict the masked token**.

The input tokens were masked in a tricky way to ***force the model to train longer but produce better results*** with three methods:

* Surprise the model by not masking a single token on $10\%$ of the dataset; for example:
```bash
The cat sat on it [because] it was a nice rug.
```
* Surprise the model by replacing the token with a random token on $10\%$ of the dataset; for example:
```bash
The cat sat on it [often] it was a nice rug.
```
* Replace a token by a [MASK] token on $80\%$ of the dataset; for example:
```bash
The cat sat on it [MASK] it was a nice rug.
```

The authors’ bold approach avoids overfitting and forces the model to train efficiently.

### **Next sentence prediction**

The second method found to train BERT is **Next Sentence Prediction (NSP)**. The input contains two sentences. In $50\%$ of the cases, the second sentence is the actual second sentence of a document. In $50\%$ of the cases, the second sentence was selected randomly and has no relation to the first one.

Two new tokens were added:

* `[CLS]` is a binary classification token added to the beginning of the first sequence to predict if the second sequence follows the first sequence. A positive sample is usually a pair of consecutive sentences taken from a dataset. A negative sample is created using sequences from different documents.
* `[SEP]` is a separation token that signals the end of a sequence.
  
For example, the input sentences taken out of a book could be:
```bash
The cat slept on the rug. It likes sleeping all day.
```

These two sentences will become one complete input sequence:
```bash
[CLS] the cat slept on the rug [SEP] it likes sleep ##ing all day[SEP]
```
This approach requires additional encoding information to distinguish sequence A from sequence B.

If we put the whole embedding process together, we obtain:

![Alt text](input_embeddings.png)

The input embeddings are obtained by summing the token embeddings, the segment (sentence, phrase, word) embeddings, and the positional encoding embeddings.

The input embedding and positional encoding sub-layer of a BERT model can be summed up as follows:

* A sequence of words is broken down into **WordPiece** tokens.
* A `[MASK]` token will randomly replace the initial word tokens for masked language modeling training.
* A `[CLS]` classification token is inserted at the beginning of a sequence for classification purposes.
* A `[SEP]` token separates two sentences (segments, phrases) for NSP training.
* Sentence embedding is added to token embedding, so that **sentence A** has a different sentence embedding value than **sentence B**.
* Positional encoding is learned. The sine-cosine positional encoding method of the original Transformer is not applied.

Some additional key features are:

* BERT uses bidirectional attention in its multi-head attention sub-layers, opening vast horizons of learning and understanding relationships between tokens.
* BERT introduces scenarios of unsupervised embedding, pretraining models with unlabeled text. Unsupervised scenarios force the model to think harder during the multi-head attention learning process. This makes BERT learn how languages are built and apply this knowledge to downstream tasks without having to pretrain each time.
* BERT also uses supervised learning, covering all bases in the pretraining process.

BERT has improved the training environment of transformers. Let’s now see the motivation of pretraining and how it helps the fine-tuning process.

## **Pretraining and fine-tuning a BERT model**

BERT is a two-step framework. The first step is pretraining, and the second is fine-tuning

![Alt text](bert_framework.png)

Training a transformer model can take hours, if not days. It takes quite some time to engineer the architecture and parameters and select the proper datasets to train a transformer model.

Pretraining is the first step of the BERT framework, which can be broken down into two sub-steps:
* Defining the model’s architecture: number of layers, number of heads, dimensions, and the other building blocks of the model
* Training the model on **MLM** and **NSP** tasks
The second step of the BERT framework is fine-tuning, which can also be broken down into two sub-steps:

* Initializing the downstream model chosen with the trained parameters of the pretrained BERT model
* Fine-tuning the parameters for specific downstream tasks such as **Recognizing Textual Entailment (RTE)**, question answering (**SQuAD v1.1**, **SQuAD v2.0)**, and **Situations With Adversarial Generations (SWAG)**

In this chapter, the BERT model we will fine-tune will be trained on **The Corpus of Linguistic Acceptability (CoLA)**. The downstream task is based on Neural Network *Acceptability Judgments by Alex Warstadt, Amanpreet Singh, and Samuel R. Bowman*.

We will fine-tune a BERT model that will determine the grammatical acceptability of a sentence. The fine-tuned model will have acquired a certain level of linguistic competence.

We have gone through BERT architecture and its pretraining and fine-tuning framework. Let’s now fine-tune a BERT model.

### **Fine-tuning BERT**

This section will fine-tune a BERT model to predict the downstream task of Acceptability Judgments and measure the predictions with the Matthews Correlation Coefficient (MCC), which will be explained in the Evaluating using Matthews Correlation Coefficient section of this chapter.

Open `BERT_Fine_Tuning_Sentence_Classification_GPU.ipynb` file